In [110]:
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

def plotter(total, theta_range_steps, function, function_args, plotting_args):
      
    data = [dict(
            visible = False,
            line=dict(color='#00CED1', width=6),
            name = 'Binomial probability',
            x = np.linspace(0,1,theta_range_steps),
            y = function(n, function_args),
            #title.text = 'Distribution P(theta) vs theta'
            ) for n in np.linspace(0,total,total+1)]           # slider steps
        
    data[5]['visible'] = True

    steps = []
    
        
    for i in range(len(data)):
        
        if (function == binomial_vector):
            step = dict(
                # Update method allows us to update both trace and layout properties
                method = 'update',  
                args = [
                    # Make the ith trace visible
                    {'visible': [t == i for t in range(len(data))]},
                    # Set the title for the ith trace
                    {'title.text': 'num_p %d, total events %d' % (i,total)}],
            )
        elif (function == beta_vector):
            step = dict(
                # Update method allows us to update both trace and layout properties
                method = 'update',  
                args = [
                    # Make the ith trace visible
                    {'visible': [t == i for t in range(len(data))]},
                    # Set the title for the ith trace
                    {'title.text': 'alpha %d, beta %d' % (i + 1, total - i + 1)}],
            )   
        elif (function == poisson_vector):
            step = dict(
                # Update method allows us to update both trace and layout properties
                method = 'update',  
                args = [
                    # Make the ith trace visible
                    {'visible': [t == i for t in range(len(data))]},
                    # Set the title for the ith trace
                    {'title.text': 'y %d' % (i)}],
            )
            
        steps.append(step)
    
    sliders = [dict(
        active = 10,
        currentvalue = {"prefix": "Number of positive events: "},
        pad = {"t": 50},
        steps = steps
    )]

    layout = dict(sliders=sliders)
    fig = dict(data=data, layout=layout)
    fig['layout']['height'] = 600
    fig['layout']['width'] = 800
    fig['layout']['title'] = plotting_args['title']
    fig['layout']['xaxis_title'] = plotting_args['title']
    return(fig)



def plotter_over_y(theta_steps, y_range_steps, function, function_args, plotting_args):
      
    data = [dict(
            visible = False,
            line=dict(color='#00CED1', width=6),
            name = 'Binomial probability',
            x = np.linspace(0,y_range_steps),
            y = function(theta, function_args),
            #title.text = 'Distribution P(theta) vs theta'
            ) for theta in np.linspace(0,theta_steps,theta_steps + 1)]         # slider steps
        
    data[5]['visible'] = True

    steps = []
    
    for i in range(len(data)):
        
        if (function == binomial_vector):
            step = dict(
                # Update method allows us to update both trace and layout properties
                method = 'update',  
                args = [
                    # Make the ith trace visible
                    {'visible': [t == i for t in range(len(data))]},
                    # Set the title for the ith trace
                    {'title.text': 'num_p %d, total events %d' % (i,total)}],
            )
        elif (function == beta_vector):
            step = dict(
                # Update method allows us to update both trace and layout properties
                method = 'update',  
                args = [
                    # Make the ith trace visible
                    {'visible': [t == i for t in range(len(data))]},
                    # Set the title for the ith trace
                    {'title.text': 'alpha %d, beta %d' % (i + 1, total - i + 1)}],
            )   
        elif (function == poisson_vector):
            step = dict(
                # Update method allows us to update both trace and layout properties
                method = 'update',  
                args = [
                    # Make the ith trace visible
                    {'visible': [t == i for t in range(len(data))]},
                    # Set the title for the ith trace
                    {'title.text': 'Mean parameter theta %d' % (i)}],
            )
            
        steps.append(step)
    
    sliders = [dict(
        active = 10,
        currentvalue = {"prefix": "Mean parameter theta: "},
        pad = {"t": 50},
        steps = steps
    )]

    layout = dict(sliders=sliders)
    fig = dict(data=data, layout=layout)
    fig['layout']['height'] = 600
    fig['layout']['width'] = 800
    fig['layout']['title'] = plotting_args['title']
    fig['layout']['xaxis_title'] = plotting_args['title']
    return(fig)



In [61]:
import numpy as np
from math import comb
import scipy
from scipy.special import gamma, factorial
import plotly.express as px

## Total is the number of total events, the number of successes or positive events have a probability theta. 
## The plot shows how p(theta) varies for various values of successes given by num_p 

In [26]:
def binomial_vector(num_p, args):
    total_events = args['total']
    theta_range_steps = args['theta_range_steps']
    theta =  np.linspace(0,1,theta_range_steps)
    ncr = comb(int(total_events), int(num_p))
    p_theta = ncr * theta**num_p * (1 - theta)**(total_events - num_p)
    return(p_theta)
   
def binomial_likelihood(num_p, total_events, theta):
    p_y_given_theta = theta**num_p * (1 - theta)**(total_events - num_p)
    return(p_y_given_theta)


fig = plotter(total=10, theta_range_steps=50, function=binomial_vector, 
              function_args={'total':10, 'theta_range_steps': 50},
              plotting_args={'title':'Binomial distribution - P(theta/y) vs theta, move slider to vary num_p'})
pio.show(fig)


### Beta distribution is the conjugate prior of a binomial distribution. A class of conjugate priors for a sampling model P(y/theta) is one that makes the posterior P(theta/y) have the same form as the prior

#### This means that the posterior, computed from a likelihood function that has a binomial form,  will have a beta distribution. If we start with a uniform prior for p(theta), this posterior can be analytically computed as beta(a, b) where
#### a corresponds to num_p + 1 = number of positive + 1
#### b corresponds to total - num_p + 1 = number of negative + 1
#### a and b are pseudo, counts
#### a = 1 and b = 1 gives you a uniform distribution
#### If a beta distribution beta(a,b) is used as a prior for a binomial sampling model, the posterior is then given by beta(a+1,b+1)
#### Conjugate priors make calculating the posterior easy, but it may not accurately reflect our prior information always

In [44]:
def beta_vector(num_p, args):
    alpha = num_p + 1
    beta = args['total'] - num_p + 1
    theta_range_steps = args['theta_range_steps']
    
    theta =  np.linspace(0,1,theta_range_steps)
    term = gamma(alpha + beta) / ( gamma(alpha) * gamma(beta) )
    p_theta = term * theta**(alpha - 1) * (1 - theta)**(beta - 1)
    return(p_theta)


fig = plotter(total=10, theta_range_steps=50, function=beta_vector, 
              function_args={'total':10, 'theta_range_steps': 50},
              plotting_args={'title': 'Beta distribution - P(theta/y) vs theta, move slider to vary num_p'})
pio.show(fig)

In [60]:
def beta_vector_individual(alpha, beta, theta_range_steps):
    theta =  np.linspace(0,1,theta_range_steps)
    term = gamma(alpha + beta) / ( gamma(alpha) * gamma(beta) )
    p_theta = term * theta**(alpha - 1) * (1 - theta)**(beta - 1)
    fig = px.line(x=theta, y=p_theta, color_discrete_sequence=["steelblue"], 
                  height=600, width=800, title=" alpha %d, beta %d" %(alpha, beta))
    fig.data[0].line['width'] = 4
    fig.show()


beta_vector_individual(2,10,100)

In [111]:
def poisson_vector(theta, args):
    y_range_steps = args['y_range_steps']
    y =  np.linspace(0,y_range_steps)
    
    p_theta = (theta**y * np.exp(-theta)) / factorial(y)
    return(p_theta)

# y_range is how far you want to compute y values
fig = plotter_over_y(theta_steps=20, y_range_steps=20, function=poisson_vector, 
              function_args={'y_range_steps': 20},
              plotting_args={'title': 'Poisson distribution - P(y/theta) vs y, move slider to vary the mean parameter theta'})
pio.show(fig)

In [109]:
theta = 18
y_range_steps = 20
y =  np.linspace(0,y_range_steps)
p_theta = (theta**y * np.exp(-theta)) / factorial(y)
fig = px.line(x=y, y=p_theta, color_discrete_sequence=["steelblue"], 
                  height=600, width=800, title="Poisson distribution")
fig.data[0].line['width'] = 4
fig.show()